In [1]:
#https://www.huduser.gov/portal/datasets/fmr.html#data_2024

import pandas as pd
import numpy as np

In [2]:
# Import raw data
df=pd.read_csv("Rent Data 2017-2024.csv" ,sep="," ,encoding="utf-8")
df.head()

,Year,stusps,state,hud_area_code,countyname,county_town_name,metro,hud_area_name,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4
0,2024,AL,1,METRO33860M33860,Autauga County,NaN,1.0,"Montgomery, AL MSA",100199999,55639.0,836,913,1092,1383,1753
1,2024,AL,1,METRO19300M19300,Baldwin County,NaN,1.0,"Daphne-Fairhope-Foley, AL MSA",100399999,218289.0,1051,1056,1362,1670,2114
2,2024,AL,1,NCNTY01005N01005,Barbour County,NaN,0.0,"Barbour County, AL",100599999,25026.0,652,656,857,1089,1141
3,2024,AL,1,METRO13820M13820,Bibb County,NaN,1.0,"Birmingham-Hoover, AL HUD Metro FMR Area",100799999,22374.0,983,1109,1245,1570,1752
4,2024,AL,1,METRO13820M13820,Blount County,NaN,1.0,"Birmingham-Hoover, AL HUD Metro FMR Area",100999999,57755.0,983,1109,1245,1570,1752


In [3]:
# Clean raw data
df = df.rename(columns={'stusps': 'State', 'state': 'State No.','countyname':'County','pop2020':'Population','fmr_0':"Studio","fmr_1":"One Bedroom","fmr_2":"Two Bedroom","fmr_3":"Three Bedroom","fmr_4":"Four Bedroom"})
df = df[['Year','State', 'County','Population','Studio','One Bedroom','Two Bedroom','Three Bedroom','Four Bedroom']]

df = df.dropna()

df["Year"] = df['Year'].astype(int)
df["Studio"] = df['Studio'].astype(float)
df["One Bedroom"] = df['One Bedroom'].astype(float)
df["Two Bedroom"] = df['Two Bedroom'].astype(float)
df["Three Bedroom"] = df['Three Bedroom'].astype(float)
df["Four Bedroom"] = df['Four Bedroom'].astype(float)
df["Population"] = df['Population'].astype(int)


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38126 entries, 0 to 38129
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           38126 non-null  int64  
 1   State          38126 non-null  object 
 2   County         38126 non-null  object 
 3   Population     38126 non-null  int64  
 4   Studio         38126 non-null  float64
 5   One Bedroom    38126 non-null  float64
 6   Two Bedroom    38126 non-null  float64
 7   Three Bedroom  38126 non-null  float64
 8   Four Bedroom   38126 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 2.9+ MB


In [4]:
# Save cleaned file
df.to_csv('clean_rent_data.csv', index=False)